In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot
from math import sqrt
from PIL import Image
import os

from pgan import PGAN, WeightedSum
from tensorflow.keras import backend

In [2]:
# Create a Keras callback that periodically saves generated images and updates alpha in WeightedSum layers
class GANMonitor(keras.callbacks.Callback):
  def __init__(self, num_img=16, latent_dim=512, prefix=''):
    self.num_img = num_img
    self.latent_dim = latent_dim
    self.random_latent_vectors = tf.random.normal(shape=[num_img, self.latent_dim], seed=9434)
    self.steps_per_epoch = 0
    self.epochs = 0
    self.steps = self.steps_per_epoch * self.epochs
    self.n_epoch = 0
    self.prefix = prefix
  
  def set_prefix(self, prefix=''):
    self.prefix = prefix
  
  def set_steps(self, steps_per_epoch, epochs):
    self.steps_per_epoch = steps_per_epoch
    self.epochs = epochs
    self.steps = self.steps_per_epoch * self.epochs

  def on_epoch_begin(self, epoch, logs=None):
    self.n_epoch = epoch


  def on_epoch_end(self, epoch, logs=None):
    samples = self.model.generator(self.random_latent_vectors)
    samples = (samples * 0.5) + 0.5
    n_grid = int(sqrt(self.num_img))

    fig, axes = pyplot.subplots(n_grid, n_grid, figsize=(4*n_grid, 4*n_grid))
    sample_grid = np.reshape(samples[:n_grid * n_grid], (n_grid, n_grid, samples.shape[1], samples.shape[2], samples.shape[3]))

    for i in range(n_grid):
      for j in range(n_grid):
        axes[i][j].set_axis_off()
        samples_grid_i_j = Image.fromarray((sample_grid[i][j] * 255).astype(np.uint8))
        samples_grid_i_j = samples_grid_i_j.resize((128,128))
        axes[i][j].imshow(np.array(samples_grid_i_j))
    title = f'images/plot_{self.prefix}_{epoch:05d}.png'
    pyplot.savefig(title, bbox_inches='tight')
    print(f'\n saved {title}')
    pyplot.close(fig)
  

  def on_batch_begin(self, batch, logs=None):
    # Update alpha in WeightedSum layers
    alpha = ((self.n_epoch * self.steps_per_epoch) + batch) / float(self.steps - 1)
    #print(f'\n {self.steps}, {self.n_epoch}, {self.steps_per_epoch}, {alpha}')
    for layer in self.model.generator.layers:
      if isinstance(layer, WeightedSum):
        backend.set_value(layer.alpha, alpha)
    for layer in self.model.discriminator.layers:
      if isinstance(layer, WeightedSum):
        backend.set_value(layer.alpha, alpha)

In [3]:

# DEFINE FILEPATH AND PARAMETERS
# can use celeb A mask dataset on https://github.com/switchablenorms/CelebAMask-HQ 
DATA_ROOT = '/home/trojan/Desktop/dimentia/PGGAN/pggan-tensorflow-main/dataset_CN/dataset' 
NOISE_DIM = 512
# Set the number of batches, epochs and steps for trainining.
# Look 800k images(16x50x1000) per each lavel
BATCH_SIZE = [16, 16, 16, 16, 16, 16, 4]
EPOCHS = 50
STEPS_PER_EPOCH = 308 #1000

In [4]:
# Normalilze [-1, 1] input images
def preprocessing_image(img):
  img = img.astype('float32')
  img = (img - 127.5) / 127.5
  return img

train_image_generator = ImageDataGenerator(horizontal_flip=True, preprocessing_function=preprocessing_image)
train_dataset = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE[0],
                                                          directory=DATA_ROOT,
                                                          shuffle=True,
                                                          target_size=(4,4),
                                                          class_mode='binary')


Found 4919 images belonging to 1 classes.


In [5]:
# Instantiate the optimizer for both networks
# learning_rate will be equalized per each layers by the WeightScaling scheme
generator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)



cbk = GANMonitor(num_img=64, latent_dim=NOISE_DIM, prefix='0_init')
cbk.set_steps(steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)

# Instantiate the PGAN(PG-GAN) model.
pgan = PGAN(
    latent_dim = NOISE_DIM, 
    d_steps = 1,
)

checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"

# Compile models
pgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
)

print(len(train_dataset))
# Draw models
tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{pgan.n_depth}.png', show_shapes=True)
tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{pgan.n_depth}.png', show_shapes=True)

# Start training the initial generator and discriminator
pgan.fit(train_dataset, steps_per_epoch = STEPS_PER_EPOCH, epochs = EPOCHS, callbacks=[cbk])
pgan.save_weights(checkpoint_path)

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
pixel_normalization (PixelNo (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              4194304   
_________________________________________________________________
weight_scaling_4 (WeightScal (None, 8192)              0         
_________________________________________________________________
bias_4 (Bias)                (None, 8192)              8192      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
pixel_normalization_1 (Pixel (None, 8192)              0 

Epoch 22/50
307/308 [============================>.] - ETA: 0s - d_loss: -0.1175 - g_loss: 0.2260
 saved images/plot_0_init_00021.png
308/308 [==============================] - 14s 47ms/step - d_loss: -0.1190 - g_loss: 0.2259
Epoch 23/50
307/308 [============================>.] - ETA: 0s - d_loss: -0.1150 - g_loss: 0.2047
 saved images/plot_0_init_00022.png
308/308 [==============================] - 15s 49ms/step - d_loss: -0.1149 - g_loss: 0.2040
Epoch 24/50
307/308 [============================>.] - ETA: 0s - d_loss: -0.1206 - g_loss: 0.2237
 saved images/plot_0_init_00023.png
308/308 [==============================] - 14s 47ms/step - d_loss: -0.1206 - g_loss: 0.2242
Epoch 25/50
307/308 [============================>.] - ETA: 0s - d_loss: -0.1090 - g_loss: 0.2190
 saved images/plot_0_init_00024.png
308/308 [==============================] - 14s 47ms/step - d_loss: -0.1099 - g_loss: 0.2205
Epoch 26/50
307/308 [============================>.] - ETA: 0s - d_loss: -0.1079 - g_loss: 0.191

In [6]:
# Train faded-in / stabilized generators and discriminators
for n_depth in range(1, 7):
  # Set current level(depth)
  pgan.n_depth = n_depth

  # Set parameters like epochs, steps, batch size and image size
  steps_per_epoch = STEPS_PER_EPOCH
  epochs = int(EPOCHS*(BATCH_SIZE[0]/BATCH_SIZE[n_depth]))
  DATA_ROOT = f'/home/trojan/Desktop/dimentia/PGGAN/pggan-tensorflow-main/dataset_CN/dataset-{n_depth}'
  train_dataset = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE[n_depth],
                                                          directory=DATA_ROOT,
                                                          shuffle=True,
                                                          target_size=(4*(2**n_depth), 4*(2**n_depth)),
                                                          class_mode='binary')
  cbk.set_prefix(prefix=f'{n_depth}_fade_in')
  cbk.set_steps(steps_per_epoch=steps_per_epoch, epochs=epochs)

  # Put fade in generator and discriminator
  pgan.fade_in_generator()
  pgan.fade_in_discriminator()

  # Draw fade in generator and discriminator
  tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{n_depth}_fade_in.png', show_shapes=True)
  tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{n_depth}_fade_in.png', show_shapes=True)

  pgan.compile(
      d_optimizer=discriminator_optimizer,
      g_optimizer=generator_optimizer,
  )
  # Train fade in generator and discriminator
  pgan.fit(train_dataset, steps_per_epoch = steps_per_epoch, epochs = epochs, callbacks=[cbk])
  # Save models
  checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"
  pgan.save_weights(checkpoint_path)

  # Change to stabilized generator and discriminator
  cbk.set_prefix(prefix=f'{n_depth}_stabilize')
  pgan.stabilize_generator()
  pgan.stabilize_discriminator()

  # Draw stabilized generator and discriminator
  tf.keras.utils.plot_model(pgan.generator, to_file=f'generator_{n_depth}_stabilize.png', show_shapes=True)
  tf.keras.utils.plot_model(pgan.discriminator, to_file=f'discriminator_{n_depth}_stabilize.png', show_shapes=True)
  pgan.compile(
      d_optimizer=discriminator_optimizer,
      g_optimizer=generator_optimizer,
  )
  # Train stabilized generator and discriminator
  pgan.fit(train_dataset, steps_per_epoch = steps_per_epoch, epochs = epochs, callbacks=[cbk])
  # Save models
  checkpoint_path = f"ckpts/pgan_{cbk.prefix}.ckpt"
  pgan.save_weights(checkpoint_path)

Found 4919 images belonging to 1 classes.
Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
pixel_normalization (PixelNorma (None, 512)          0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 8192)         4194304     pixel_normalization[0][0]        
__________________________________________________________________________________________________
weight_scaling_4 (WeightScaling (None, 8192)         0           dense_1[0][0]                    
________________________________________________

leaky_re_lu_10 (LeakyReLU)      (None, 8, 8, 512)    0           bias_13[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 4, 4, 512)    0           bias[1][0]                       
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 4, 4, 512)    0           leaky_re_lu_10[0][0]             
__________________________________________________________________________________________________
weighted_sum_1 (WeightedSum)    (None, 4, 4, 512)    1           leaky_re_lu[1][0]                
                                                                 average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
minibatch_stdev (MinibatchStdev (None, 4, 4, 513)    0           weighted_sum_1[0][0]             
__________

Epoch 20/50
308/308 [==============================] - ETA: 0s - d_loss: -0.1053 - g_loss: 0.3308
 saved images/plot_1_fade_in_00019.png
308/308 [==============================] - 27s 88ms/step - d_loss: -0.1058 - g_loss: 0.3316
Epoch 21/50
308/308 [==============================] - ETA: 0s - d_loss: -0.1133 - g_loss: 0.3394
 saved images/plot_1_fade_in_00020.png
308/308 [==============================] - 27s 87ms/step - d_loss: -0.1134 - g_loss: 0.3404
Epoch 22/50
308/308 [==============================] - ETA: 0s - d_loss: -0.1059 - g_loss: 0.3226
 saved images/plot_1_fade_in_00021.png
308/308 [==============================] - 27s 88ms/step - d_loss: -0.1066 - g_loss: 0.3243
Epoch 23/50
308/308 [==============================] - ETA: 0s - d_loss: -0.1068 - g_loss: 0.2677
 saved images/plot_1_fade_in_00022.png
308/308 [==============================] - 27s 87ms/step - d_loss: -0.1067 - g_loss: 0.2693
Epoch 24/50
308/308 [==============================] - ETA: 0s - d_loss: -0.1027 - g

308/308 [==============================] - ETA: 0s - d_loss: -0.0295 - g_loss: 0.1014
 saved images/plot_1_stabilize_00000.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0294 - g_loss: 0.1009
Epoch 2/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0381 - g_loss: 0.1248
 saved images/plot_1_stabilize_00001.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0376 - g_loss: 0.1267
Epoch 3/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0280 - g_loss: 0.1524
 saved images/plot_1_stabilize_00002.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0277 - g_loss: 0.1497
Epoch 4/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0281 - g_loss: 0.0799
 saved images/plot_1_stabilize_00003.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0278 - g_loss: 0.0812
Epoch 5/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0262 - g_loss: 0

Epoch 37/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0285 - g_loss: 0.1079
 saved images/plot_1_stabilize_00036.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0286 - g_loss: 0.1092
Epoch 38/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0171 - g_loss: 0.1058
 saved images/plot_1_stabilize_00037.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0171 - g_loss: 0.1033
Epoch 39/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0216 - g_loss: 0.1594
 saved images/plot_1_stabilize_00038.png
308/308 [==============================] - 26s 84ms/step - d_loss: -0.0217 - g_loss: 0.1589
Epoch 40/50
308/308 [==============================] - ETA: 0s - d_loss: -0.0204 - g_loss: 0.0293
 saved images/plot_1_stabilize_00039.png
308/308 [==============================] - 26s 85ms/step - d_loss: -0.0202 - g_loss: 0.0286
Epoch 41/50
308/308 [==============================] - ETA: 0s - d_loss: -0.

...

Epoch 35/200
308/308 [==============================] - ETA: 0s - d_loss: -6.3187 - g_loss: 9.2942
 saved images/plot_6_stabilize_00034.png
308/308 [==============================] - 169s 548ms/step - d_loss: -6.2835 - g_loss: 9.2983
Epoch 36/200
308/308 [==============================] - ETA: 0s - d_loss: -5.9259 - g_loss: 8.2927
 saved images/plot_6_stabilize_00035.png
308/308 [==============================] - 169s 549ms/step - d_loss: -5.9001 - g_loss: 8.3104
Epoch 37/200
308/308 [==============================] - ETA: 0s - d_loss: -6.2589 - g_loss: 7.7926
 saved images/plot_6_stabilize_00036.png
308/308 [==============================] - 169s 548ms/step - d_loss: -6.2748 - g_loss: 7.8901
Epoch 38/200
308/308 [==============================] - ETA: 0s - d_loss: -6.4526 - g_loss: 9.1847
 saved images/plot_6_stabilize_00037.png
308/308 [==============================] - 2030s 7s/step - d_loss: -6.4112 - g_loss: 8.8671
Epoch 39/200
308/308 [==============================] - ETA: 0s - 

308/308 [==============================] - ETA: 0s - d_loss: -7.0174 - g_loss: 10.2499
 saved images/plot_6_stabilize_00069.png
308/308 [==============================] - 169s 548ms/step - d_loss: -7.0598 - g_loss: 10.2645
Epoch 71/200
308/308 [==============================] - ETA: 0s - d_loss: -7.2965 - g_loss: 10.0512
 saved images/plot_6_stabilize_00070.png
308/308 [==============================] - 168s 547ms/step - d_loss: -7.2463 - g_loss: 9.9991
Epoch 72/200
308/308 [==============================] - ETA: 0s - d_loss: -6.1316 - g_loss: 8.8285
 saved images/plot_6_stabilize_00071.png
308/308 [==============================] - 183s 593ms/step - d_loss: -6.1159 - g_loss: 8.7685
Epoch 73/200
308/308 [==============================] - ETA: 0s - d_loss: -5.6830 - g_loss: 8.7631
 saved images/plot_6_stabilize_00072.png
308/308 [==============================] - 169s 548ms/step - d_loss: -5.7109 - g_loss: 8.7853
Epoch 74/200
308/308 [==============================] - ETA: 0s - d_loss: 

Epoch 105/200
308/308 [==============================] - ETA: 0s - d_loss: -7.5158 - g_loss: 10.3917
 saved images/plot_6_stabilize_00104.png
308/308 [==============================] - 169s 549ms/step - d_loss: -7.4874 - g_loss: 10.4227
Epoch 106/200
308/308 [==============================] - ETA: 0s - d_loss: -7.9074 - g_loss: 10.2938
 saved images/plot_6_stabilize_00105.png
308/308 [==============================] - 169s 547ms/step - d_loss: -7.9141 - g_loss: 10.2464
Epoch 107/200
308/308 [==============================] - ETA: 0s - d_loss: -7.0233 - g_loss: 9.6010
 saved images/plot_6_stabilize_00106.png
308/308 [==============================] - 169s 547ms/step - d_loss: -6.9968 - g_loss: 9.5826
Epoch 108/200
308/308 [==============================] - ETA: 0s - d_loss: -8.1071 - g_loss: 10.6951
 saved images/plot_6_stabilize_00107.png
308/308 [==============================] - 170s 553ms/step - d_loss: -8.1116 - g_loss: 10.5626
Epoch 109/200
308/308 [==============================]

Epoch 140/200
308/308 [==============================] - ETA: 0s - d_loss: -7.8866 - g_loss: 10.1838
 saved images/plot_6_stabilize_00139.png
308/308 [==============================] - 169s 549ms/step - d_loss: -7.8889 - g_loss: 10.2347
Epoch 141/200
308/308 [==============================] - ETA: 0s - d_loss: -7.9681 - g_loss: 11.0115
 saved images/plot_6_stabilize_00140.png
308/308 [==============================] - 169s 548ms/step - d_loss: -7.9387 - g_loss: 11.0244
Epoch 142/200
308/308 [==============================] - ETA: 0s - d_loss: -8.5131 - g_loss: 11.6960
 saved images/plot_6_stabilize_00141.png
308/308 [==============================] - 168s 547ms/step - d_loss: -8.5292 - g_loss: 11.6875
Epoch 143/200
308/308 [==============================] - ETA: 0s - d_loss: -8.0111 - g_loss: 8.8507
 saved images/plot_6_stabilize_00142.png
308/308 [==============================] - 169s 547ms/step - d_loss: -8.0450 - g_loss: 8.8624
Epoch 144/200
308/308 [==============================]

Epoch 175/200
308/308 [==============================] - ETA: 0s - d_loss: -9.0689 - g_loss: 11.9859
 saved images/plot_6_stabilize_00174.png
308/308 [==============================] - 169s 547ms/step - d_loss: -9.0749 - g_loss: 12.0239
Epoch 176/200
308/308 [==============================] - ETA: 0s - d_loss: -8.7588 - g_loss: 11.9236
 saved images/plot_6_stabilize_00175.png
308/308 [==============================] - 169s 549ms/step - d_loss: -8.7543 - g_loss: 11.9584
Epoch 177/200
308/308 [==============================] - ETA: 0s - d_loss: -9.0311 - g_loss: 12.6518
 saved images/plot_6_stabilize_00176.png
308/308 [==============================] - 169s 547ms/step - d_loss: -9.0142 - g_loss: 12.6095
Epoch 178/200
308/308 [==============================] - ETA: 0s - d_loss: -8.5826 - g_loss: 11.7281
 saved images/plot_6_stabilize_00177.png
308/308 [==============================] - 169s 548ms/step - d_loss: -8.5704 - g_loss: 11.7177
Epoch 179/200
308/308 [=============================